In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm, max_token_limit=120, return_messages=True, memory_key="history"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


# RunnablePassthrough을 사용하여 필요한 데이터를 먼저 load하고 그 값을 history라는 이름으로 prompt에 전달한다.
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


invoke_chain("My name is nico")
invoke_chain("What is my name?")

In [ ]:
from langchain.schema import BaseOutputParser


# 받은 문자열을 ,을 기준으로 리스트로 만든다.
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        # list 각 항목에 다시 str.strip 함수를 적용
        # str.strip 함수는 각 문자열의 앞 뒤 공백을 제거한다.
        return list(map(str.strip, items))


parser = CommaOutputParser()

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everythig you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# chain은 PromptTemplate | Model | OutputParser 로 만들 수 있다.
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "what are animals?"})

In [ ]:
from langchain.llms.openai import OpenAI  # 기본값으로 davinchi-003를사용
from langchain.chat_models import ChatOpenAI  # 기본값으로 GTP3.5-Turbo를 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

# streaming=True, callbacks=[StreamingStdOutCallbackHandler()] 옵션을 넣어주면 실시간 출력을 볼 수 있다.
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# 답번의 예시
examples = [
    {
        "question": "What do you know about Japan?",
        "answer": """
        I know this:
        Capital: Tokyo
        Languae: Japanese
        Food: Sushi
        Currency: Yen
        """,
    }
]

# 답변 양식에 대한 prompt
example_prompt = PromptTemplate.from_template("Human:{question}|nAI:{answer}")

# 질문에 대한 prompt 작성
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # suffix가 유저의 질문 부분임
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Korea")
# 결과: 'Human:What do you know about Japan?|nAI:\n  I know this:\n  Capital: Tokyo\n  Languae: Japanese\n  Food: Sushi\n   :Currency: Yen\n   \n\nHuman: What do you know about Korea?'

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

# 랭체인을 사용해 얻은 값을 자동으로 메모리에 캐싱하도록 한다.
set_llm_cache(SQLiteCache("cache.db"))

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everythig you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

# 다음 chain은 PromptTemplate | Model | OutputParser 형태로 이루어져 있다.
chain = template | chat | CommaOutputParser()
# 결과: ['Mammals', 'Birds', 'Reptiles', 'Amphibians', 'Fish']
chain.invoke({"max_items": 5, "question": "what are animals?"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

# get_openai_callback()를 사용하면 openAI를 사용한 결과를 받아올 수 있다.(챗 결과랑 다른거임)
with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    # 위 처리에 소요된 비용을 출력
    print(usage)

In [ ]:
from langchain.llms.openai import OpenAI  # 기본값으로 davinchi-003를사용
from langchain.chat_models import ChatOpenAI  # 기본값으로 GTP3.5-Turbo를 사용
from langchain.schema import HumanMessage, AIMessage, SystemMessage, BaseOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# streaming=True, callbacks=[StreamingStdOutCallbackHandler()] 옵션을 넣어주면 실시간 출력을 볼 수 있다.
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "Japan",
        "answer": """
        I know this:
        Capital: Tokyo
        Languae: Japanese
        Food: Sushi
        :Currency: Yen
        """,
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "What do you know about {country}?"), ("ai", "{answer}")]
)


example_prompt = FewShotChatMessagePromptTemplate(
    # ChatPromptTemplate을 사용해서 답변 양식을 만든다.
    example_prompt=example_prompt,
    # 실제 답변의 예시를 만든다.
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert"),
        # 이전에 니가 이런식으로 답변했다는 것을 example_prompt로 예를 들어준다.
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Lee, I live in South Korea", "Wow, that is so cool!")
add_message("South Korea is getting fallen.", "what? why?")

get_history()

In [ ]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm LEE, I live in South Korea", "Wow that is so cool!")
memory.load_memory_variables({"input": "who is LEE"})

In [ ]:
add_message("LEE likes kimchi", "Wow that is so cool!")
memory.load_memory_variables({"inputs": "what does LEE like"})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./files/textbook.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


# 해당 함수에서 돌려주는 값은 context에 들어가는 값이기 때문에 하나의 string을 반환해야한다.
# 그렇기 때문에 inputs에 들어 있는 모든 데이터를 하나의
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]

    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )


# documents: vectorstore에서 받은 Documents
# question: vectorstore에서 값을 받기 위한 질문
# RunnableLambda를 사용하면 chain의 어느 위치에서도 함수 실행 가능
# RunnableLambda에서 map_docs 함수를 실행하는데 자동으로 map_docs에 필요한 파라미터로써 앞에서 실시한 "documents": retriever, "question": RunnablePassthrough() 값이 들어간다.
# 참고로 "documents": retriever, "question": RunnablePassthrough()은 사전형 데이터로 inputs 파라미터에 들어간다.
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 요약받은 Doument(=context)에 prompt를 적용시켜 llm을 한다.
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("Where does Winston live?")

In [ ]:
from gpt4all import GPT4All
model = GPT4All(model_name='orca-mini-3b-gguf2-q4_0.gguf')
with model.chat_session():
    response1 = model.generate(prompt='hello', temp=0)
    response2 = model.generate(prompt='write me a short poem', temp=0)
    response3 = model.generate(prompt='thank you', temp=0)
    print()

    for message in model.current_chat_session:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print(f"SYSTEM:  {content}")
        elif role == 'user':
            print(f"USER: {content}")
        else:
            print(f"ASSISTANT: {content}")
            print("-----------")
            print("")
        


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    # STRUCTURED를 사용하면 파라미터가 여러 개인 func을 지정할 수 있다.
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # 여러 개의 Tool을 지정할 수 있다.
    tools=[
        # StructuredTool를 사용하면 파라미터가 여러 개인 func을 지정할 수 있다.
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers.",
        ),
    ],
)

prompt = "Tell me Total Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

# pydantic를 사용하여 래핑할 클래스를 생성
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    # Tool에서 args_schema를 정의하면 LangChain이 자동으로 인수의 유효성을 검사하고 API를 생성한다.
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    # 타입으로 OPENAI_FUNCTIONS를 사용
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [16]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.schema import SystemMessage
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    # 덕덕고에서 검색하기 위한 검색어
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )

# 덕덕고로 검색 실시하는 툴
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    # 종목 티커
    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL,TSLA",
    )

# 종목 티커를 사용해서 종목 개요 데이터를 가져오는 Tool
class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        # 회사 개요 데이터
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()

# 종목 티커를 사용해서 손익계산서 데이터를 가져오는 Tool
class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        # 손익계산서 데이터
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]

# 종목 티커를 사용해서 주가(주간) 데이터를 가져오는 Tool
class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        # 주간 주가 데이터
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        response = r.json()
        # items() 함수를 사용해서 (key-value)로 된 list를 얻는다.
        # 80주간의 주가 데이터만 가져오게 한다.
        return list(response["Weekly Time Series"].items())[:80]


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    # 4개의 툴을 전부 넣는다.
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
    # OpenAI 에이전트에도 다음과 같은 방법으로 프롬프트를 커스텀할 수 있다.
    agent_kwargs={
        "system_message": SystemMessage(content="""
            You are a hedge fund manager.
            
            You evaluate a company and provide your opinion and reasons why the stock is a buy or not.
            
            Consider the performance of a stock, the company overview and the income statement.
            
            Be assertive in your judgement and recommend the stock or advise the user against it.
        """)
    }
)

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


The latest Cloudflare stock prices, stock quotes, news, and NET history to help you invest and trade smarter. ... Cloudflare's market capitalization is $32.66 B by 298.38 M shares outstanding. Is ... Stock analysis for Cloudflare Inc (NET:New York) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Cloudflare Stock Surges on Strong Earnings, Upbeat Outlook—Key Price Level to Watch. Shares in Cloudflare (NET) surged nearly 25% in premarket trading Friday after the cybersecurity and content delivery solutions provider delivered a better-than-expected quarterly report and issued fi... 6 weeks ago - Investopedia. Cloudflare Stock Is Surging. A high-level overview of Cloudflare, Inc. (NET) stock. Stay up to date on the latest stock price, chart, news, analysis, fundamentals, trading and investment tools. Real-t

{'input': "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment.",
 'output': "Cloudflare Inc. (NET) is a technology company that operates a cloud platform offering a range of network services to companies worldwide. Here's an analysis of Cloudflare's financials, income statements, and stock performance to help you evaluate its potential as an investment:\n\n### Company Overview:\n- **Market Capitalization:** $32.66 billion\n- **Sector:** Technology\n- **Industry:** Services - Prepackaged Software\n- **52-Week High:** $116\n- **52-Week Low:** $39.90\n- **Shares Outstanding:** 298.38 million\n- **Revenue TTM:** $1.3 billion\n- **Profit Margin:** -14.2%\n- **Operating Margin TTM:** -11.8%\n- **Return on Assets TTM:** -4.34%\n- **Return on Equity TTM:** -26.5%\n- **Analyst Target Price:** $101.73\n- **Forward P/E Ratio:** 163.93\n- **Price to Sales Ratio TTM:** 25.21\n- 

In [12]:
import yfinance as yf

# 각 데이터가 어떤 의미인지 적혀있음 - https://recheeze.tistory.com/55
msft = yf.Ticker("005930.KS") # 삼성전자

# 일일 주가(기간: 1달)
hist = msft.history(period="1mo")

# 손익계산서(반기)
msft.income_stmt
# 손익계산서(분기)
msft.quarterly_income_stmt

# 재무재표(반기)
msft.balance_sheet
# 재무재표(분기)
msft.quarterly_balance_sheet

# 현금 흐름표
msft.cashflow
msft.quarterly_cashflow

# 뉴스 - 그렇게 도움 되지 않음 -> 직접 구글링으로 검색하는게 나을듯
msft.news

[{'uuid': '92ab7c59-df30-3d54-8e9c-03a3abb57d34', 'title': "Stock market today: Asia markets are mixed after Wall Street's strong manufacturing data", 'publisher': 'Associated Press Finance', 'link': 'https://finance.yahoo.com/news/stock-market-today-asia-markets-074253682.html', 'providerPublishTime': 1712043773, 'type': 'STORY', 'thumbnail': {'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/ySU7laMJIDNAZpPiEbTreA--~B/aD0yMDMyO3c9MzA0ODthcHBpZD15dGFjaHlvbg--/https://media.zenfs.com/en/ap_finance_articles_694/eb9ee5f67dce20d85b2243f989e789e2', 'width': 3048, 'height': 2032, 'tag': 'original'}, {'url': 'https://s.yimg.com/uu/api/res/1.2/qpxYRHBIQ0I7s2fXTqLMRQ--~B/Zmk9ZmlsbDtoPTE0MDtweW9mZj0wO3c9MTQwO2FwcGlkPXl0YWNoeW9u/https://media.zenfs.com/en/ap_finance_articles_694/eb9ee5f67dce20d85b2243f989e789e2', 'width': 140, 'height': 140, 'tag': '140x140'}]}, 'relatedTickers': ['^GSPC']}, {'uuid': '3816fd85-30cf-37a0-8ad9-27c55a9c7840', 'title': 'Prepayments (CK93) - Totalkredit A/S'

In [17]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
)
# SQLDatabase를 불러온다
db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

# SQLDatabaseToolkit 사용
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    # OPENAI_FUNCTIONS를 사용해도 괜찮고 ZERO_SHOT_REACT_DESCRIPTION를 사용해도 괜찮음
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent.invoke(
    "Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue."
)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


directors, movies
Invoking: `sql_db_schema` with `movies`



CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	revenue	title	vote_average	vote_count	overview	tagline	uid	director_id
43597	Avatar	237000000	150	2009-12-10	2787965087	Avatar	7.2	11800	In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but 	Enter the World of Pandora.	19995	4762
43598	Pirates of the Caribbean: At World's End	300000000	139	2007-05-19	961000000	Pirates of the Caribbean: At World's End	6.9	4500	Captain Barbossa, long believed to be dead, has come back to

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'The movies with the highest votes but the lowest budgets are:\n\n1. Movie: Inception\n   - Budget: $160,000,000\n   - Vote Count: 13,752\n   - Director ID: 4765\n   - Revenue: $825,532,764\n\n2. Movie: The Dark Knight\n   - Budget: $185,000,000\n   - Vote Count: 12,002\n   - Director ID: 4765\n   - Revenue: $1,004,558,444\n\n3. Movie: Avatar\n   - Budget: $237,000,000\n   - Vote Count: 11,800\n   - Director ID: 4762\n   - Revenue: $2,787,965,087\n\n4. Movie: The Avengers\n   - Budget: $220,000,000\n   - Vote Count: 11,776\n   - Director ID: 4769\n   - Revenue: $1,519,557,910\n\n5. Movie: Deadpool\n   - Budget: $58,000,000\n   - Vote Count: 10,995\n   - Director ID: 5151\n   - Revenue: $783,112,979\n\n6. Movie: Interstellar\n   - Budget: $165,000,000\n   - Vote Count: 10,867\n   - Director ID: 4765\n   - Revenue: $675,12